In [1]:
import os
import cv2
import json
import pickle
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from shapely import geometry, ops
from tqdm import tqdm

import rasterio
import rasterio.features
from geopy.distance import distance

import cosmic_eye_client
import orbital_vault as ov
from CustomerDatabase import CustomerDatabase
from datetime import datetime, date
from pimsys.regions.RegionsDb import RegionsDb

In [2]:
import torch
import clip
from PIL import Image as imp
from typing import Dict, Any

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model, preprocess = clip.load("ViT-B/32", device=device)
# model, preprocess = clip.load("ViT-L/14@336px", device=device)

def clip_similarity(
    model: Any,
    preprocess: Any,
    image: np.ndarray,
    image_before: np.ndarray,
    device: torch.device,
) -> np.ndarray:
    """Computes the similarity between two images using CLIP model

    Args:
        model (any): model used by CLIP
        preprocess (any): preprocessing function used by CLIP
        image (Image): image to be compared
        image_before (Image): reference image to be compared with for similarity

    Returns:
        float: similarity between the two images (0-1)
    """

    image_p = preprocess(imp.fromarray(image)).unsqueeze(0).to(device)
    image_before_p = preprocess(imp.fromarray(image_before)).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features1 = model.encode_image(image_p)
        image_features2 = model.encode_image(image_before_p)

    image_features1 /= image_features1.norm(dim=-1, keepdim=True)
    image_features2 /= image_features2.norm(dim=-1, keepdim=True)
    similarity = image_features2.cpu().numpy() @ image_features1.cpu().numpy().T

    return similarity


In [3]:
def download_from_geoserve(image, region, width, height, auth=None):

    arguments = {"layer_name": image["wms_layer_name"], "bbox": "%s,%s,%s,%s" % (region[0], region[1], region[2], region[3]), "width": width, "height": height,}

    if "image_url" in image.keys() and image["image_url"] is not None:
        image_url = image["image_url"]
        if not image_url[-1] == "?":
            image_url = image_url + "?"
        arguments["bbox"] = "%s,%s,%s,%s" % (region[1], region[0], region[3], region[2],)
        url = (image_url + "&VERSION=1.3.0&SERVICE=WMS&REQUEST=GetMap&STYLES=&CRS=epsg:4326&BBOX=%(bbox)s&WIDTH=%(width)s&HEIGHT=%(height)s&FORMAT=image/png&LAYERS=%(layer_name)s" % arguments)

    try:
        resp = requests.get(url, auth=auth)
        image = np.asarray(bytearray(resp.content), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        image = image[:, :, [2, 1, 0]]
    except:
        print(resp)
        print(url)
        return None
    
    return image 

def download_from_mapserver(image, region, width, height, auth=None):
    url = "https://maps.orbitaleye.nl/mapserver/?map=/maps/_"
    image_url = url + f'{image["wms_layer_name"]}.map&VERSION=1.0.0&SERVICE=WMS&REQUEST=GetMap&STYLES=&SRS=epsg:4326&BBOX={region[0]},{region[1]},{region[2]},{region[3]}&WIDTH={width}&HEIGHT={height}&FORMAT=image/png&LAYERS={image["wms_layer_name"]}'
    resp = requests.get(image_url, auth=auth)
    try:
        image = np.asarray(bytearray(resp.content), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        image = image[:, :, [2, 1, 0]]
    except:
        print(image_url)
        return None
    
    return image


def download_image(wms_image, indicator_window, width, height, creds_mapserver, creds_geoserve):
    if wms_image['downloader'] == 'geoserve':
        if creds_geoserve is None:
            image = download_from_geoserve(wms_image, indicator_window.bounds, width, height)#, auth=(creds_geoserve['user'], creds_geoserve['password']))
        else:
            image = download_from_geoserve(wms_image, indicator_window.bounds, width, height, auth=(creds_geoserve['user'], creds_geoserve['password']))
    else:
        image = download_from_mapserver(wms_image, indicator_window.bounds, width, height, auth=(creds_mapserver['username'], creds_mapserver['password']))
    return image

def get_mask_valid_area(wms_image, window, width, height):
    valid_area = wms_image['valid_area']
    geotiff_transform = rasterio.transform.from_bounds(*window.bounds, width, height)

    mask_valid = rasterio.features.rasterize([valid_area], out_shape=(width, height), transform=geotiff_transform).astype(bool)
    
    return mask_valid

def download_images(wms_images, tpi_region, width, height, creds_mapserver, creds_geoserve):
    for w in wms_images:
        w['area'] = w['valid_area'].intersection(tpi_region).area
        
    wms_images = sorted(wms_images, key=lambda d: d['area'])
    
    # wms_image_after = wms_images_after[0]
    image_out = np.zeros((width,height,3),dtype=np.uint8)
    for wms_image in wms_images:
        image = download_image(wms_image, tpi_region, width, height, creds_mapserver, creds_geoserve)
        mask = get_mask_valid_area(wms_image, tpi_region, width, height)
        image_out[mask] = image[mask]
        
    return image_out

def get_wms_images(database, coords, tpi_timestamp1, tpi_timestamp2):
    wms_images = database.get_optical_images_containing_point_in_period(coords, [tpi_timestamp1, tpi_timestamp2])

    wms_images = [x for x in wms_images if x['source'] != 'Sentinel-2']
    wms_images = [x for x in wms_images if x['user_classification'] in ['GOOD', 'OK', '', None]]
    # images = [x for x in images if x['pixel_resolution_x'] == 0.3]
    wms_images = sorted(wms_images, key=lambda x: x['capture_timestamp'])
    return wms_images

def get_bbox(coords, image_buffer):
    image_buffer = 0.128
    tpi_buffer_north = distance(kilometers=image_buffer).destination((coords[1], coords[0]), bearing=0)
    tpi_buffer_east = distance(kilometers=image_buffer).destination((coords[1], coords[0]), bearing=90)
    vertical_buffer = tpi_buffer_north[0] - coords[1]
    horizontal_buffer = tpi_buffer_east[1] - coords[0]
    tpi_north = coords[1] + vertical_buffer
    tpi_east = coords[0] + horizontal_buffer
    tpi_south = coords[1] - vertical_buffer
    tpi_west = coords[0] - horizontal_buffer
    tpi_region = geometry.Polygon([[tpi_east, tpi_north], [tpi_west, tpi_north], [tpi_west, tpi_south],[tpi_east, tpi_south], [tpi_east, tpi_north]])
    return tpi_region

def group_by_hour(data):
    # Sort data by capture_timestamp
    # data.sort(key=lambda x: x['capture_timestamp'])
    
    grouped_data = []
    current_group = []
    current_interval_start = None

    for item in data:
        timestamp = item['capture_timestamp']
        
        if current_interval_start is None:
            current_interval_start = timestamp
        
        if timestamp < current_interval_start + timedelta(hours=1):
            current_group.append(item)
        else:
            grouped_data.append(current_group)
            current_group = [item]
            current_interval_start = timestamp
    
    if current_group:
        grouped_data.append(current_group)
    
    return grouped_data

In [4]:
def get_images_for_tpi_ml(settings, tpi, creds_mapserver, creds_geoserve):
    with RegionsDb(settings['settings_db']) as database:
        coords = tpi['point']
        #tpi_poly = geometry.Polygon(tpi['coordinates']).buffer(0.)
        tpi_timestamp_after  = tpi['referencedate2']
        tpi_timestamp_before = tpi['referencedate1']
        # tpi_date = datetime.fromtimestamp(tpi_timestamp)
        tpi_region = get_bbox(coords, 0.128)
        width, height = 512, 512

        wms_images_after = get_wms_images(database, coords, tpi_timestamp_after-3000, tpi_timestamp_after+3000)

        if len(wms_images_after) == 0:
            # print(1)
            return None
            # return None, None, None, None

        # wms_image_after = wms_images_after[-1]

        wms_images_before = get_wms_images(database, coords, tpi_timestamp_before-3000, tpi_timestamp_before+3000)#[::-1]

        if len(wms_images_before) == 0:
            # print(1)
            return None
            # return None, None, None, None

        # wms_image_before = wms_images_before[-1]
        
        image_after = download_images(wms_images_after, tpi_region, width, height, creds_mapserver, creds_geoserve)
        # image_after = download_image(wms_image_after, tpi_region, width, height, creds_mapserver, creds_geoserve)
        if image_after is None:
            return None
        
        image_before = download_images(wms_images_before, tpi_region, width, height, creds_mapserver, creds_geoserve)        
        if image_before is None:
            return None
        
        out = {"image_before": image_before, "image_after": image_after, "poly": tpi_region, "tpi": tpi}

        return out
    
def get_images_for_tpi(settings, tpi, model, preprocess, creds_mapserver, creds_geoserve):
    with RegionsDb(settings['settings_db']) as database:
        coords = tpi['point']
        #tpi_poly = geometry.Polygon(tpi['coordinates']).buffer(0.)
        tpi_timestamp = tpi['referencedate2']
        tpi_date = datetime.fromtimestamp(tpi_timestamp)
        tpi_region = get_bbox(coords, 0.128)
        width, height = 512, 512

        wms_images_after = get_wms_images(database, coords, tpi_timestamp-3000, tpi_timestamp+3000)

        if len(wms_images_after) == 0:
            # print(1)
            return None
            # return None, None, None, None

        # wms_image_after = wms_images_after[-1]

        wms_images_before = get_wms_images(database, coords, tpi_timestamp-360*24*60*60, tpi_timestamp-6*24*60*60)
        wms_images_before = group_by_hour(wms_images_before)[::-1][:5]

        if len(wms_images_before) == 0:
            # print(1)
            return None
            # return None, None, None, None

        image_after = download_images(wms_images_after, tpi_region, width, height, creds_mapserver, creds_geoserve)
        if image_after is None:
            return None
        
        images_before = [download_images(w, tpi_region, width, height, creds_mapserver, creds_geoserve) for w in wms_images_before]
        images_before = [im for im in images_before if im is not None]
        
        score_clip = [clip_similarity(model, preprocess, image_after, im, device) for im in images_before]

        id_max = np.argmax(score_clip)
        image_before = images_before[id_max]

        out = {"image_before": image_before, "image_after": image_after, "score": max(score_clip), "poly": tpi_region, "tpi": tpi}

        return out
    

In [5]:
# start_date = datetime(2024,10,15)
# end_date   = datetime(2024,11,1)
start_date = datetime(2024,8,1)
end_date   = datetime(2024,11,1)

customer_db_creds = ov.get_customerdb_credentials()
customer_db = CustomerDatabase(customer_db_creds['username'], customer_db_creds['password'])

settings = {}
settings['settings_db'] = ov.get_sarccdb_credentials()

# project_name = 'Gasunie-DE-2024'
project_name = 'NGC-2023'

In [6]:

project = customer_db.get_project_by_name(project_name)

ce_login = ov.get_project_server_credentials(project.get("name"), project.get("servers")[0].get("name"))
client = cosmic_eye_client.connect(project.get("servers")[0].get("domain"))
client.login(ce_login['user'], ce_login['password'])

filters = {
    'bounding_box': [[180., -90.], [180., 90.], [-180., 90.], [-180., -90.]],
    'daterange': [start_date.timestamp(), end_date.timestamp()],
    }
# user_made_tpis = client.getUserMadeTpisSpaceTime(filters=filters)
user_made_tpis = client.getUserMadeTpisSpaceTime(filters)
# tpis_optical_2 = [t for t in user_made_tpis if t['source'] == 'optical' and t['confirmation_factor'] == 2]
tpis_optical_ml_temp = [t for t in user_made_tpis if t['source'] == 'optical (ml)']

tpis_optical_ml = client.getUserMadeTpis([x['id'] for x in tpis_optical_ml_temp])
# for t in tpis_optical_2:
#     t['project_name'] = project_name
#     t['project_id'] = project['id']
for t in tpis_optical_ml:
    t['project_name'] = project_name
    t['project_id'] = project['id']

print(project_name, len(tpis_optical_ml))

total_tpis = tpis_optical_ml

NGC-2023 2231


In [10]:
# for x in tpis_optical_ml:
#     if x['confirmation_factor'] == 2:
#         x['notes'] = 'good'

# tpis_optical_ml = [x for x in tpis_optical_ml if 'Orbital Eye' not in x['notes'] and 'from' not in x['notes']]

for x in tpis_optical_ml:
    if x['confirmation_factor'] == 2:
        x['notes'] = 'relevant'
    else:
        x['notes'] = 'not relevant'


In [11]:
df_train = tpis_optical_ml

np.unique([x['notes'] for x in tpis_optical_ml])

array(['not relevant', 'relevant'], dtype='<U12')

In [12]:
len(tpis_optical_ml)

2231

In [13]:
import os

output_dataset_directory = './fa_benchmark_test_ngc/'

# os.makedirs(output_dataset_directory+'train/', exist_ok=True)
# os.makedirs(output_dataset_directory+'val/', exist_ok=True)
# os.makedirs(output_dataset_directory+'test/', exist_ok=True)

In [14]:

counter = 0

project = customer_db.get_project_by_name(project_name)

if not project['flags']['use_geoserve_hosting']:# or 'PTT' in project_name:
    creds_mapserver = ov.get_image_broker_credentials()
    creds_geoserve = None
else:
    creds_geoserve = ov.get_geoserve_credentials(project_name)
    creds_mapserver = ov.get_image_broker_credentials()


for i in tqdm(range(len(df_train))):
    t = df_train[i]
    # t['point'] = json.loads(t['point'])
    # t['coordinates'] = json.loads(t['coordinates'])
    t['referencedate2'] = int(t['referencedate2'])
    if t['referencedate1'] is not None and~np.isnan(t['referencedate1']):
        t['referencedate1'] = int(t['referencedate1'])
    else:
        t['referencedate1'] = None

    out = None
    if t['source'] == 'optical':
        if t['referencedate1'] is None:
            out = get_images_for_tpi(settings, t, model, preprocess, creds_mapserver, creds_geoserve)
        else:
            out = get_images_for_tpi_ml(settings, t, creds_mapserver, creds_geoserve)
    else:
        if t['referencedate1'] is None:
            out = get_images_for_tpi(settings, t, model, preprocess, creds_mapserver, creds_geoserve)
        else:
            out = get_images_for_tpi_ml(settings, t, creds_mapserver, creds_geoserve)
        # out = get_images_for_tpi_ml(settings, t, creds_mapserver, creds_geoserve)

    if out is not None:
        with open(output_dataset_directory+'data_tpi_'+project_name+'_'+str(out['tpi']['id'])+'.p', 'wb') as f:
            pickle.dump(out, f)
        counter += 1
print(project_name, counter)
    # break

100%|██████████| 2231/2231 [1:34:28<00:00,  2.54s/it]  

NGC-2023 2184
